In [5]:
import sys
import os
import timeit

current_dir = os.getcwd()

parent_dir = os.path.dirname(current_dir)
sys.path.append(parent_dir)

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm import *
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from matplotlib import gridspec
import random

from Networks.DNN import PINN
from Networks.IA_DNN import Attention_PINN
from Networks.Fourier_DNN import Fourier_PINN
from Networks.AF_PINN import AF_PINN
from Networks.Efficient_KAN import KAN
from Networks.Cheby_KAN import ChebyKAN
from DataGenerator import DataGenerator

import pandas as pd

from torch.utils.data import TensorDataset, DataLoader

import time
from datetime import datetime

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("Using device:", device)
def seed_torch(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
seed_torch(42)

Using device: cuda:0


In [6]:
L = 600.0
h = 100.0
anneal_time= 0.1
D_A = 360.0  #6nm^2/s 360nm^2/min
k11,k12,k21 = 0.0,0.0,0.0
N_SA,N_SB,N_SC,N_SAB,N_SABB,N_SAAB = 90.0,48.0,90.0,90.0,90.0,90.0

geom = [0,L]
time = [0,anneal_time]

##How I genertate data
generator = DataGenerator(geom,time)

from torch._tensor import Tensor
from pde_system import compute_pde, compute_bc, compute_ic,relative_error
from graphics import plot_loss_curve,plot_relative_concen,plot_3d_concen
# from Utilitiy.residual_adaptive_resample import select_high_loss_points

def get_points(N:list) -> tuple[Tensor, Tensor, Tensor, Tensor]: 
    coords_pde =  generator.LHS_generator(N[0], points_type = 'domain' , nondim= True) 
    coords_bc1 =  generator.LHS_generator(N[1], points_type = 'left_bc' , nondim= True) 
    coords_bc2 =  generator.LHS_generator(N[1], points_type = 'right_bc' , nondim= True) 
    coords_ic =  generator.LHS_generator(N[2], points_type = 'initial' , nondim= True) 
    return coords_pde, coords_bc1, coords_bc2, coords_ic

N = [15000, 3000, 4000]

from torch import Tensor


hiddens = [3]  ##[3,4] already finished
neurons = [64]

activation = torch.nn.functional.tanh

start_lr = 1e-3
end_lr = 1e-6

weight = [1, 1, 1, 1, 1, 1]

mode = "AF_PINN"
for hidden in hiddens:
    # for neuron in neurons:
        for annealing_time in [60, 10, 0.1]:  ##['PINN', 'IA_PINN', 'Fourier_PINN', 'AF_PINN']

            if annealing_time == 0.1:
                neuron = 64
            elif annealing_time == 10:
                neuron = 96
            else:
                neuron = 128

            layer = [2] + [neuron]*hidden + [6]
            
            star_time = timeit.default_timer()
            num_epochs = 2000
            sub_spochs = 5

            loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic = [],[],[],[]
            torch.cuda.empty_cache()
            print(f'{mode}_[{neuron}]x{hidden}:')
            
            if mode == 'PINN':
                model = PINN(layer,activation,non_negative=True).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'IA_PINN':
                model = Attention_PINN(layer,activation,non_negative=True).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'Fourier_PINN':
                model = Fourier_PINN(layer,non_negative=True, activation= activation, use_rff=True, rff_num_features=2*neuron, rff_sigma=1.0).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'AF_PINN':
                model = AF_PINN(layer, non_negative=True,activation = activation, use_rff=True, rff_num_features=2*neuron, rff_sigma=1.0).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            else:
                print("Man! What can I say! Wrong Mode!!!")
                break            
            points_types = ['domain', 'left_bc', 'right_bc', 'initial']
            coords_pde, coords_bc1, coords_bc2, coords_ic = get_points(N)    
            # coords_pde, coords_bc1, coords_bc2, coords_ic = [
            #     generator.grid_generator(2500 if pt in ['left_bc', 'right_bc'] else 10201, points_type=pt, nondim= True) 
            #     for pt in points_types
            # ]
            # Since x = [0,200] is interphase-boundary area, we should add my sample points in this area.
            # area_enhance = generator.LHS_local_enhance(N = 5000, points_type='domain',enhance_area=[[0,200],[0,60]],nondim= True)

            # coords_pde = torch.cat((coords_pde, area_enhance), dim=0)

            for idx in range(sub_spochs):
                print(f'Epoch: {idx+1}/{sub_spochs}----------------')
                model.train()
                
                pbar = tqdm(range(num_epochs))
                for epoch in pbar:
                    current_lr = optimizer.param_groups[0]['lr'] 
                    def closure() -> Tensor:
                        global loss_pde, D_star, loss_bc1, loss_bc2, loss_ic1, loss_ic2
                        optimizer.zero_grad()

                        # 计算损失时使用batch数据
                        loss_eq1, loss_eq2, loss_eq3, loss_eq4, loss_eq5, loss_eq6, D_star = compute_pde(model, coords_pde, anneal_time=anneal_time)
                        loss_pde = loss_eq1 + loss_eq2 + loss_eq3 + loss_eq4 + loss_eq5 + loss_eq6
                        loss_bc1 = compute_bc(model, coords_bc1)
                        loss_bc2 = compute_bc(model, coords_bc2)
                        loss_ic1, loss_ic2 = compute_ic(model, coords_ic)

                        #Give a weight to initial loss term
                        loss = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type= 2)
                        return loss

                    optimizer.step(closure)
                    
                    # 如果使用scheduler（只在Adam阶段）
                    if scheduler is not None:
                        scheduler.step()
                    
                    loss_total = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

                    loss_log_total.append(loss_total.item())
                    loss_log_pde.append(loss_pde.item())
                    loss_log_bc.append(loss_bc1.item() + loss_bc2.item())
                    loss_log_ic.append(loss_ic1.item() + loss_ic2.item())
                    
                    if epoch % 10 == 0:
                        pbar.set_postfix({
                            'Loss': '{0:.3e}'.format(loss_total),
                            'PDE': '{0:.3e}'.format(weight[0]*loss_pde.item()),
                            'BC': '{0:.3e},{1:.3e}'.format(weight[1]*loss_bc1.item(),weight[2]*loss_bc2.item()),
                            'IC': '{0:.3e},{1:.3e}'.format(weight[3]*loss_ic1.item(),weight[4]*loss_ic2.item()),
                            'D*': '{0:.2e},{1:.2e}'.format(torch.max(D_star),torch.min(D_star)),
                            'lr': '{0:.2e}'.format(current_lr),
                        })        
                    # # 将四个张量放在一个列表中并随机打乱
                    shuffled_tensors = [tensor[torch.randperm(tensor.size(0))] for tensor in [coords_pde, coords_bc1, coords_bc2, coords_ic]]

                    # 解包到四个变量中
                    coords_pde, coords_bc1, coords_bc2, coords_ic = shuffled_tensors
                
                    if torch.isnan(loss_total):
                        print('Loss is Nan, please check your model parameters')
                        break
                
                true_path = './FDM/range[0,1]_01/'
                file_names = ['u1_output.csv','u2_output.csv','u3_output.csv','u4_output.csv','u5_output.csv','u6_output.csv']
                true_data = [pd.read_csv(true_path + file_name,header=None) for file_name in file_names]

                model.eval()
                val_data = generator.grid_generator(N=10201, points_type= 'domain', nondim= True)
                result = model(val_data).data.cpu().numpy()
                relative_error(result,true_data)

                ##模型储存
                save_path = f'./Results/{mode}_[{neuron}]x{hidden}/model_save/'
                os.makedirs(save_path, exist_ok=True)  # 确保保存路径存在
                # 指定模型文件名
                model_file = os.path.join(save_path, f'{mode}_[{neuron}]x{hidden}_{(idx+1)*num_epochs}.pth')
                # 保存模型
                torch.save(model.state_dict(), model_file)
                print(f"模型已保存至 {model_file}")
            
            # 绘制结果
            plot_loss_curve(f'{mode}_[{neuron}]x{hidden}',loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic)
            
            plot_relative_concen(f'{mode}_[{neuron}]x{hidden}',result,data_save=True, annealing_time= annealing_time)
            
            plot_3d_concen(f'{mode}_[{neuron}]x{hidden}', result, true_data)

            Training_time = timeit.default_timer() - star_time
            print(f'Training time: {Training_time:.2f}s-------------------------')


AF_PINN_[128]x3:
Epoch: 1/5----------------


100%|██████████| 2000/2000 [10:29<00:00,  3.18it/s, Loss=3.281e-03, PDE=1.855e-04, BC=6.155e-04,4.393e-06, IC=2.466e-03,9.314e-06, D*=9.99e-05,8.52e-07, lr=9.06e-04]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.034, 'SiC': 0.01, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.086, 'SiC': 0.063, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/AF_PINN_[128]x3/model_save/AF_PINN_[128]x3_2000.pth
Epoch: 2/5----------------


100%|██████████| 2000/2000 [10:11<00:00,  3.27it/s, Loss=4.225e-03, PDE=2.001e-04, BC=1.012e-05,2.725e-05, IC=3.968e-03,1.942e-05, D*=1.00e-04,6.93e-07, lr=6.56e-04]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.035, 'SiC': 0.012, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.114, 'SiC': 0.084, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/AF_PINN_[128]x3/model_save/AF_PINN_[128]x3_4000.pth
Epoch: 3/5----------------


...

R^2 Error: {'Ni': 0.016, 'SiC': 0.003, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.051, 'SiC': 0.023, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/AF_PINN_[96]x3/model_save/AF_PINN_[96]x3_10000.pth


e:\Report 12 Some Try\Main_Diffusion_Reaction\graphics.py:137: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\graphics.py:138: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


Training time: 2396.82s-------------------------
AF_PINN_[64]x3:
Epoch: 1/5----------------


 39%|███▉      | 779/2000 [02:19<03:39,  5.57it/s, Loss=8.909e-03, PDE=1.019e-03, BC=6.289e-04,1.568e-04, IC=7.086e-03,1.839e-05, D*=9.94e-05,1.61e-06, lr=9.85e-04]


KeyboardInterrupt: 

In [2]:
L = 600.0
h = 100.0
anneal_time= 0.1
D_A = 360.0  #6nm^2/s 360nm^2/min
k11,k12,k21 = 0.0,0.0,0.0
N_SA,N_SB,N_SC,N_SAB,N_SABB,N_SAAB = 90.0,48.0,90.0,90.0,90.0,90.0

geom = [0,L]
time = [0,anneal_time]

##How I genertate data
generator = DataGenerator(geom,time)

from torch._tensor import Tensor
from pde_system import compute_pde, compute_bc, compute_ic,relative_error
from graphics import plot_loss_curve,plot_relative_concen,plot_3d_concen
# from Utilitiy.residual_adaptive_resample import select_high_loss_points

def get_points(N:list) -> tuple[Tensor, Tensor, Tensor, Tensor]: 
    coords_pde =  generator.LHS_generator(N[0], points_type = 'domain' , nondim= True) 
    coords_bc1 =  generator.LHS_generator(N[1], points_type = 'left_bc' , nondim= True) 
    coords_bc2 =  generator.LHS_generator(N[1], points_type = 'right_bc' , nondim= True) 
    coords_ic =  generator.LHS_generator(N[2], points_type = 'initial' , nondim= True) 
    return coords_pde, coords_bc1, coords_bc2, coords_ic

N = [15000, 3000, 4000]

from torch import Tensor


hiddens = [3]  ##[3,4] already finished
neurons = [64]

activation = torch.nn.functional.tanh

start_lr = 1e-3
end_lr = 1e-6

weight = [1, 1, 1, 1, 1, 1]
for hidden in hiddens:
    for neuron in neurons:
        for mode in ['PINN', 'IA_PINN','Fourier_PINN', 'AF_PINN']:  ##['PINN', 'IA_PINN', 'Fourier_PINN', 'AF_PINN']

            layer = [2] + [neuron]*hidden + [6]
            
            star_time = timeit.default_timer()
            num_epochs = 2000
            sub_spochs = 5

            loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic = [],[],[],[]
            torch.cuda.empty_cache()
            print(f'{mode}_[{neuron}]x{hidden}:')
            
            if mode == 'PINN':
                model = PINN(layer,activation,non_negative=True).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'IA_PINN':
                model = Attention_PINN(layer,activation,non_negative=True).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'Fourier_PINN':
                model = Fourier_PINN(layer,non_negative=True, activation= activation, use_rff=True, rff_num_features=2*neuron, rff_sigma=1.0).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'AF_PINN':
                model = AF_PINN(layer, non_negative=True,activation = activation, use_rff=True, rff_num_features=2*neuron, rff_sigma=1.0).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            else:
                print("Man! What can I say! Wrong Mode!!!")
                break            
            points_types = ['domain', 'left_bc', 'right_bc', 'initial']
            coords_pde, coords_bc1, coords_bc2, coords_ic = get_points(N)    
            # coords_pde, coords_bc1, coords_bc2, coords_ic = [
            #     generator.grid_generator(2500 if pt in ['left_bc', 'right_bc'] else 10201, points_type=pt, nondim= True) 
            #     for pt in points_types
            # ]
            # Since x = [0,200] is interphase-boundary area, we should add my sample points in this area.
            # area_enhance = generator.LHS_local_enhance(N = 5000, points_type='domain',enhance_area=[[0,200],[0,60]],nondim= True)

            # coords_pde = torch.cat((coords_pde, area_enhance), dim=0)

            for idx in range(sub_spochs):
                print(f'Epoch: {idx+1}/{sub_spochs}----------------')
                model.train()
                
                pbar = tqdm(range(num_epochs))
                for epoch in pbar:
                    current_lr = optimizer.param_groups[0]['lr'] 
                    def closure() -> Tensor:
                        global loss_pde, D_star, loss_bc1, loss_bc2, loss_ic1, loss_ic2
                        optimizer.zero_grad()

                        # 计算损失时使用batch数据
                        loss_eq1, loss_eq2, loss_eq3, loss_eq4, loss_eq5, loss_eq6, D_star = compute_pde(model, coords_pde, anneal_time=anneal_time)
                        loss_pde = loss_eq1 + loss_eq2 + loss_eq3 + loss_eq4 + loss_eq5 + loss_eq6
                        loss_bc1 = compute_bc(model, coords_bc1)
                        loss_bc2 = compute_bc(model, coords_bc2)
                        loss_ic1, loss_ic2 = compute_ic(model, coords_ic)

                        #Give a weight to initial loss term
                        loss = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type= 2)
                        return loss

                    optimizer.step(closure)
                    
                    # 如果使用scheduler（只在Adam阶段）
                    if scheduler is not None:
                        scheduler.step()
                    
                    loss_total = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

                    loss_log_total.append(loss_total.item())
                    loss_log_pde.append(loss_pde.item())
                    loss_log_bc.append(loss_bc1.item() + loss_bc2.item())
                    loss_log_ic.append(loss_ic1.item() + loss_ic2.item())
                    
                    if epoch % 10 == 0:
                        pbar.set_postfix({
                            'Loss': '{0:.3e}'.format(loss_total),
                            'PDE': '{0:.3e}'.format(weight[0]*loss_pde.item()),
                            'BC': '{0:.3e},{1:.3e}'.format(weight[1]*loss_bc1.item(),weight[2]*loss_bc2.item()),
                            'IC': '{0:.3e},{1:.3e}'.format(weight[3]*loss_ic1.item(),weight[4]*loss_ic2.item()),
                            'D*': '{0:.2e},{1:.2e}'.format(torch.max(D_star),torch.min(D_star)),
                            'lr': '{0:.2e}'.format(current_lr),
                        })        
                    # # 将四个张量放在一个列表中并随机打乱
                    shuffled_tensors = [tensor[torch.randperm(tensor.size(0))] for tensor in [coords_pde, coords_bc1, coords_bc2, coords_ic]]

                    # 解包到四个变量中
                    coords_pde, coords_bc1, coords_bc2, coords_ic = shuffled_tensors
                
                    if torch.isnan(loss_total):
                        print('Loss is Nan, please check your model parameters')
                        break
                
                true_path = './FDM/range[0,1]_01/'
                file_names = ['u1_output.csv','u2_output.csv','u3_output.csv','u4_output.csv','u5_output.csv','u6_output.csv']
                true_data = [pd.read_csv(true_path + file_name,header=None) for file_name in file_names]

                model.eval()
                val_data = generator.grid_generator(N=10201, points_type= 'domain', nondim= True)
                result = model(val_data).data.cpu().numpy()
                relative_error(result,true_data)

                ##模型储存
                save_path = f'./Results/{mode}_[{neuron}]x{hidden}/model_save/'
                os.makedirs(save_path, exist_ok=True)  # 确保保存路径存在
                # 指定模型文件名
                model_file = os.path.join(save_path, f'{mode}_[{neuron}]x{hidden}_{(idx+1)*num_epochs}.pth')
                # 保存模型
                torch.save(model.state_dict(), model_file)
                print(f"模型已保存至 {model_file}")
            
            # 绘制结果
            plot_loss_curve(f'{mode}_[{neuron}]x{hidden}',loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic)
            
            plot_relative_concen(f'{mode}_[{neuron}]x{hidden}',result,data_save=True)
            
            plot_3d_concen(f'{mode}_[{neuron}]x{hidden}', result, true_data)

            Training_time = timeit.default_timer() - star_time
            print(f'Training time: {Training_time:.2f}s-------------------------')


PINN_[64]x3:
Epoch: 1/5----------------


100%|██████████| 2000/2000 [02:12<00:00, 15.12it/s, Loss=1.432e-02, PDE=9.445e-04, BC=4.911e-05,3.095e-06, IC=1.326e-02,6.264e-05, D*=9.93e-05,1.88e-06, lr=9.06e-04]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.123, 'SiC': 0.043, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.286, 'SiC': 0.32, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/PINN_[64]x3/model_save/PINN_[64]x3_2000.pth
Epoch: 2/5----------------


100%|██████████| 2000/2000 [02:11<00:00, 15.22it/s, Loss=6.225e-03, PDE=9.221e-04, BC=6.550e-06,1.454e-06, IC=5.285e-03,1.026e-05, D*=9.97e-05,6.50e-07, lr=6.56e-04]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.06, 'SiC': 0.017, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.152, 'SiC': 0.14, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/PINN_[64]x3/model_save/PINN_[64]x3_4000.pth
Epoch: 3/5----------------


...

R^2 Error: {'Ni': 0.019, 'SiC': 0.006, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.053, 'SiC': 0.041, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/AF_PINN_[64]x3/model_save/AF_PINN_[64]x3_8000.pth
Epoch: 5/5----------------


100%|██████████| 2000/2000 [06:12<00:00,  5.37it/s, Loss=1.035e-03, PDE=9.419e-05, BC=8.012e-07,1.484e-07, IC=9.360e-04,4.012e-06, D*=9.99e-05,4.20e-07, lr=1.00e-06]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.019, 'SiC': 0.005, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.054, 'SiC': 0.04, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/AF_PINN_[64]x3/model_save/AF_PINN_[64]x3_10000.pth


e:\Report 12 Some Try\Main_Diffusion_Reaction\graphics.py:137: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\graphics.py:138: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


Training time: 1867.30s-------------------------


In [3]:
L = 600.0
h = 100.0
anneal_time= 10
D_A = 360.0  #6nm^2/s 360nm^2/min
k11,k12,k21 = 0.0,0.0,0.0
N_SA,N_SB,N_SC,N_SAB,N_SABB,N_SAAB = 90.0,48.0,90.0,90.0,90.0,90.0

geom = [0,L]
time = [0,anneal_time]

##How I genertate data
generator = DataGenerator(geom,time)

from torch._tensor import Tensor
from pde_system import compute_pde, compute_bc, compute_ic,relative_error
from graphics import plot_loss_curve,plot_relative_concen,plot_3d_concen
# from Utilitiy.residual_adaptive_resample import select_high_loss_points

def get_points(N:list) -> tuple[Tensor, Tensor, Tensor, Tensor]: 
    coords_pde =  generator.LHS_generator(N[0], points_type = 'domain' , nondim= True) 
    coords_bc1 =  generator.LHS_generator(N[1], points_type = 'left_bc' , nondim= True) 
    coords_bc2 =  generator.LHS_generator(N[1], points_type = 'right_bc' , nondim= True) 
    coords_ic =  generator.LHS_generator(N[2], points_type = 'initial' , nondim= True) 
    return coords_pde, coords_bc1, coords_bc2, coords_ic

N = [15000, 3000, 4000]

from torch import Tensor


hiddens = [3]  ##[3,4] already finished
neurons = [96]

activation = torch.nn.functional.tanh

start_lr = 1e-3
end_lr = 1e-6

weight = [1, 1, 1, 1, 1, 1]
for hidden in hiddens:
    for neuron in neurons:
        for mode in ['PINN', 'IA_PINN','Fourier_PINN', 'AF_PINN']:  ##['PINN', 'IA_PINN', 'Fourier_PINN', 'AF_PINN']

            layer = [2] + [neuron]*hidden + [6]
            
            star_time = timeit.default_timer()
            num_epochs = 2000
            sub_spochs = 5

            loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic = [],[],[],[]
            torch.cuda.empty_cache()
            print(f'{mode}_[{neuron}]x{hidden}:')
            
            if mode == 'PINN':
                model = PINN(layer,activation,non_negative=True).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'IA_PINN':
                model = Attention_PINN(layer,activation,non_negative=True).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'Fourier_PINN':
                model = Fourier_PINN(layer,non_negative=True, activation= activation, use_rff=True, rff_num_features=2*neuron, rff_sigma=1.0).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'AF_PINN':
                model = AF_PINN(layer, non_negative=True,activation = activation, use_rff=True, rff_num_features=2*neuron, rff_sigma=1.0).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            else:
                print("Man! What can I say! Wrong Mode!!!")
                break            
            points_types = ['domain', 'left_bc', 'right_bc', 'initial']
            coords_pde, coords_bc1, coords_bc2, coords_ic = get_points(N)    
            # coords_pde, coords_bc1, coords_bc2, coords_ic = [
            #     generator.grid_generator(2500 if pt in ['left_bc', 'right_bc'] else 10201, points_type=pt, nondim= True) 
            #     for pt in points_types
            # ]
            # Since x = [0,200] is interphase-boundary area, we should add my sample points in this area.
            # area_enhance = generator.LHS_local_enhance(N = 5000, points_type='domain',enhance_area=[[0,200],[0,60]],nondim= True)

            # coords_pde = torch.cat((coords_pde, area_enhance), dim=0)

            for idx in range(sub_spochs):
                print(f'Epoch: {idx+1}/{sub_spochs}----------------')
                model.train()
                
                pbar = tqdm(range(num_epochs))
                for epoch in pbar:
                    current_lr = optimizer.param_groups[0]['lr'] 
                    def closure() -> Tensor:
                        global loss_pde, D_star, loss_bc1, loss_bc2, loss_ic1, loss_ic2
                        optimizer.zero_grad()

                        # 计算损失时使用batch数据
                        loss_eq1, loss_eq2, loss_eq3, loss_eq4, loss_eq5, loss_eq6, D_star = compute_pde(model, coords_pde, anneal_time=anneal_time)
                        loss_pde = loss_eq1 + loss_eq2 + loss_eq3 + loss_eq4 + loss_eq5 + loss_eq6
                        loss_bc1 = compute_bc(model, coords_bc1)
                        loss_bc2 = compute_bc(model, coords_bc2)
                        loss_ic1, loss_ic2 = compute_ic(model, coords_ic)

                        #Give a weight to initial loss term
                        loss = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type= 2)
                        return loss

                    optimizer.step(closure)
                    
                    # 如果使用scheduler（只在Adam阶段）
                    if scheduler is not None:
                        scheduler.step()
                    
                    loss_total = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

                    loss_log_total.append(loss_total.item())
                    loss_log_pde.append(loss_pde.item())
                    loss_log_bc.append(loss_bc1.item() + loss_bc2.item())
                    loss_log_ic.append(loss_ic1.item() + loss_ic2.item())
                    
                    if epoch % 10 == 0:
                        pbar.set_postfix({
                            'Loss': '{0:.3e}'.format(loss_total),
                            'PDE': '{0:.3e}'.format(weight[0]*loss_pde.item()),
                            'BC': '{0:.3e},{1:.3e}'.format(weight[1]*loss_bc1.item(),weight[2]*loss_bc2.item()),
                            'IC': '{0:.3e},{1:.3e}'.format(weight[3]*loss_ic1.item(),weight[4]*loss_ic2.item()),
                            'D*': '{0:.2e},{1:.2e}'.format(torch.max(D_star),torch.min(D_star)),
                            'lr': '{0:.2e}'.format(current_lr),
                        })        
                    # # 将四个张量放在一个列表中并随机打乱
                    shuffled_tensors = [tensor[torch.randperm(tensor.size(0))] for tensor in [coords_pde, coords_bc1, coords_bc2, coords_ic]]

                    # 解包到四个变量中
                    coords_pde, coords_bc1, coords_bc2, coords_ic = shuffled_tensors
                
                    if torch.isnan(loss_total):
                        print('Loss is Nan, please check your model parameters')
                        break
                
                true_path = './FDM/range[0,1]_10/'
                file_names = ['u1_output.csv','u2_output.csv','u3_output.csv','u4_output.csv','u5_output.csv','u6_output.csv']
                true_data = [pd.read_csv(true_path + file_name,header=None) for file_name in file_names]

                model.eval()
                val_data = generator.grid_generator(N=10201, points_type= 'domain', nondim= True)
                result = model(val_data).data.cpu().numpy()
                relative_error(result,true_data)

                ##模型储存
                save_path = f'./Results/{mode}_[{neuron}]x{hidden}/model_save/'
                os.makedirs(save_path, exist_ok=True)  # 确保保存路径存在
                # 指定模型文件名
                model_file = os.path.join(save_path, f'{mode}_[{neuron}]x{hidden}_{(idx+1)*num_epochs}.pth')
                # 保存模型
                torch.save(model.state_dict(), model_file)
                print(f"模型已保存至 {model_file}")
            
            # 绘制结果
            plot_loss_curve(f'{mode}_[{neuron}]x{hidden}',loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic)
            
            plot_relative_concen(f'{mode}_[{neuron}]x{hidden}',result,data_save=True)
            
            plot_3d_concen(f'{mode}_[{neuron}]x{hidden}', result, true_data)

            Training_time = timeit.default_timer() - star_time
            print(f'Training time: {Training_time:.2f}s-------------------------')


PINN_[96]x3:
Epoch: 1/5----------------


100%|██████████| 2000/2000 [02:18<00:00, 14.45it/s, Loss=3.209e-02, PDE=3.589e-03, BC=2.065e-03,2.416e-05, IC=2.613e-02,2.848e-04, D*=9.97e-03,5.35e-04, lr=9.06e-04]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.117, 'SiC': 0.023, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.129, 'SiC': 0.075, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/PINN_[96]x3/model_save/PINN_[96]x3_2000.pth
Epoch: 2/5----------------


100%|██████████| 2000/2000 [02:17<00:00, 14.51it/s, Loss=1.458e-02, PDE=2.203e-03, BC=1.156e-04,4.016e-06, IC=1.213e-02,1.307e-04, D*=9.99e-03,2.42e-04, lr=6.56e-04]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.062, 'SiC': 0.014, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.077, 'SiC': 0.048, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/PINN_[96]x3/model_save/PINN_[96]x3_4000.pth
Epoch: 3/5----------------


...

R^2 Error: {'Ni': 0.018, 'SiC': 0.005, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.024, 'SiC': 0.018, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/AF_PINN_[96]x3/model_save/AF_PINN_[96]x3_8000.pth
Epoch: 5/5----------------


100%|██████████| 2000/2000 [09:08<00:00,  3.64it/s, Loss=1.741e-03, PDE=1.196e-04, BC=1.194e-06,1.205e-07, IC=1.603e-03,1.752e-05, D*=1.00e-02,1.50e-04, lr=1.00e-06]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.016, 'SiC': 0.005, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.022, 'SiC': 0.019, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/AF_PINN_[96]x3/model_save/AF_PINN_[96]x3_10000.pth


e:\Report 12 Some Try\Main_Diffusion_Reaction\graphics.py:137: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\graphics.py:138: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


Training time: 2586.36s-------------------------


In [4]:
L = 600.0
h = 100.0
anneal_time= 60
D_A = 360.0  #6nm^2/s 360nm^2/min
k11,k12,k21 = 0.0,0.0,0.0
N_SA,N_SB,N_SC,N_SAB,N_SABB,N_SAAB = 90.0,48.0,90.0,90.0,90.0,90.0

geom = [0,L]
time = [0,anneal_time]

##How I genertate data
generator = DataGenerator(geom,time)

from torch._tensor import Tensor
from pde_system import compute_pde, compute_bc, compute_ic,relative_error
from graphics import plot_loss_curve,plot_relative_concen,plot_3d_concen
# from Utilitiy.residual_adaptive_resample import select_high_loss_points

def get_points(N:list) -> tuple[Tensor, Tensor, Tensor, Tensor]: 
    coords_pde =  generator.LHS_generator(N[0], points_type = 'domain' , nondim= True) 
    coords_bc1 =  generator.LHS_generator(N[1], points_type = 'left_bc' , nondim= True) 
    coords_bc2 =  generator.LHS_generator(N[1], points_type = 'right_bc' , nondim= True) 
    coords_ic =  generator.LHS_generator(N[2], points_type = 'initial' , nondim= True) 
    return coords_pde, coords_bc1, coords_bc2, coords_ic

N = [15000, 3000, 4000]

from torch import Tensor


hiddens = [3]  ##[3,4] already finished
neurons = [128]

activation = torch.nn.functional.tanh

start_lr = 1e-3
end_lr = 1e-6

weight = [1, 1, 1, 1, 1, 1]
for hidden in hiddens:
    for neuron in neurons:
        for mode in ['PINN', 'IA_PINN','Fourier_PINN', 'AF_PINN']:  ##['PINN', 'IA_PINN', 'Fourier_PINN', 'AF_PINN']

            layer = [2] + [neuron]*hidden + [6]
            
            star_time = timeit.default_timer()
            num_epochs = 2000
            sub_spochs = 5

            loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic = [],[],[],[]
            torch.cuda.empty_cache()
            print(f'{mode}_[{neuron}]x{hidden}:')
            
            if mode == 'PINN':
                model = PINN(layer,activation,non_negative=True).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'IA_PINN':
                model = Attention_PINN(layer,activation,non_negative=True).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'Fourier_PINN':
                model = Fourier_PINN(layer,non_negative=True, activation= activation, use_rff=True, rff_num_features=2*neuron, rff_sigma=1.0).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            elif mode == 'AF_PINN':
                model = AF_PINN(layer, non_negative=True,activation = activation, use_rff=True, rff_num_features=2*neuron, rff_sigma=1.0).to(device)
                optimizer = optim.Adam(model.parameters(), lr=start_lr)  
                scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
            else:
                print("Man! What can I say! Wrong Mode!!!")
                break            
            points_types = ['domain', 'left_bc', 'right_bc', 'initial']
            coords_pde, coords_bc1, coords_bc2, coords_ic = get_points(N)    
            # coords_pde, coords_bc1, coords_bc2, coords_ic = [
            #     generator.grid_generator(2500 if pt in ['left_bc', 'right_bc'] else 10201, points_type=pt, nondim= True) 
            #     for pt in points_types
            # ]
            # Since x = [0,200] is interphase-boundary area, we should add my sample points in this area.
            # area_enhance = generator.LHS_local_enhance(N = 5000, points_type='domain',enhance_area=[[0,200],[0,60]],nondim= True)

            # coords_pde = torch.cat((coords_pde, area_enhance), dim=0)

            for idx in range(sub_spochs):
                print(f'Epoch: {idx+1}/{sub_spochs}----------------')
                model.train()
                
                pbar = tqdm(range(num_epochs))
                for epoch in pbar:
                    current_lr = optimizer.param_groups[0]['lr'] 
                    def closure() -> Tensor:
                        global loss_pde, D_star, loss_bc1, loss_bc2, loss_ic1, loss_ic2
                        optimizer.zero_grad()

                        # 计算损失时使用batch数据
                        loss_eq1, loss_eq2, loss_eq3, loss_eq4, loss_eq5, loss_eq6, D_star = compute_pde(model, coords_pde, anneal_time=anneal_time)
                        loss_pde = loss_eq1 + loss_eq2 + loss_eq3 + loss_eq4 + loss_eq5 + loss_eq6
                        loss_bc1 = compute_bc(model, coords_bc1)
                        loss_bc2 = compute_bc(model, coords_bc2)
                        loss_ic1, loss_ic2 = compute_ic(model, coords_ic)

                        #Give a weight to initial loss term
                        loss = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

                        loss.backward()
                        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type= 2)
                        return loss

                    optimizer.step(closure)
                    
                    # 如果使用scheduler（只在Adam阶段）
                    if scheduler is not None:
                        scheduler.step()
                    
                    loss_total = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

                    loss_log_total.append(loss_total.item())
                    loss_log_pde.append(loss_pde.item())
                    loss_log_bc.append(loss_bc1.item() + loss_bc2.item())
                    loss_log_ic.append(loss_ic1.item() + loss_ic2.item())
                    
                    if epoch % 10 == 0:
                        pbar.set_postfix({
                            'Loss': '{0:.3e}'.format(loss_total),
                            'PDE': '{0:.3e}'.format(weight[0]*loss_pde.item()),
                            'BC': '{0:.3e},{1:.3e}'.format(weight[1]*loss_bc1.item(),weight[2]*loss_bc2.item()),
                            'IC': '{0:.3e},{1:.3e}'.format(weight[3]*loss_ic1.item(),weight[4]*loss_ic2.item()),
                            'D*': '{0:.2e},{1:.2e}'.format(torch.max(D_star),torch.min(D_star)),
                            'lr': '{0:.2e}'.format(current_lr),
                        })        
                    # # 将四个张量放在一个列表中并随机打乱
                    shuffled_tensors = [tensor[torch.randperm(tensor.size(0))] for tensor in [coords_pde, coords_bc1, coords_bc2, coords_ic]]

                    # 解包到四个变量中
                    coords_pde, coords_bc1, coords_bc2, coords_ic = shuffled_tensors
                
                    if torch.isnan(loss_total):
                        print('Loss is Nan, please check your model parameters')
                        break
                
                true_path = './FDM/range[0,1]_60/'
                file_names = ['u1_output.csv','u2_output.csv','u3_output.csv','u4_output.csv','u5_output.csv','u6_output.csv']
                true_data = [pd.read_csv(true_path + file_name,header=None) for file_name in file_names]

                model.eval()
                val_data = generator.grid_generator(N=10201, points_type= 'domain', nondim= True)
                result = model(val_data).data.cpu().numpy()
                relative_error(result,true_data)

                ##模型储存
                save_path = f'./Results/{mode}_[{neuron}]x{hidden}/model_save/'
                os.makedirs(save_path, exist_ok=True)  # 确保保存路径存在
                # 指定模型文件名
                model_file = os.path.join(save_path, f'{mode}_[{neuron}]x{hidden}_{(idx+1)*num_epochs}.pth')
                # 保存模型
                torch.save(model.state_dict(), model_file)
                print(f"模型已保存至 {model_file}")
            
            # 绘制结果
            plot_loss_curve(f'{mode}_[{neuron}]x{hidden}',loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic)
            
            plot_relative_concen(f'{mode}_[{neuron}]x{hidden}',result,data_save=True)
            
            plot_3d_concen(f'{mode}_[{neuron}]x{hidden}', result, true_data)

            Training_time = timeit.default_timer() - star_time
            print(f'Training time: {Training_time:.2f}s-------------------------')


PINN_[128]x3:
Epoch: 1/5----------------


100%|██████████| 2000/2000 [02:39<00:00, 12.56it/s, Loss=4.222e-02, PDE=4.181e-03, BC=4.214e-03,2.073e-04, IC=3.273e-02,8.907e-04, D*=5.99e-02,6.87e-03, lr=9.06e-04]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.125, 'SiC': 0.023, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.109, 'SiC': 0.051, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/PINN_[128]x3/model_save/PINN_[128]x3_2000.pth
Epoch: 2/5----------------


100%|██████████| 2000/2000 [02:39<00:00, 12.55it/s, Loss=2.042e-02, PDE=2.729e-03, BC=1.509e-04,6.703e-05, IC=1.682e-02,6.567e-04, D*=5.99e-02,5.80e-03, lr=6.56e-04]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.084, 'SiC': 0.014, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.078, 'SiC': 0.03, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/PINN_[128]x3/model_save/PINN_[128]x3_4000.pth
Epoch: 3/5----------------


...

R^2 Error: {'Ni': 0.034, 'SiC': 0.009, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.031, 'SiC': 0.019, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/AF_PINN_[128]x3/model_save/AF_PINN_[128]x3_8000.pth
Epoch: 5/5----------------


100%|██████████| 2000/2000 [11:31<00:00,  2.89it/s, Loss=3.683e-03, PDE=3.654e-04, BC=7.672e-06,8.269e-07, IC=3.189e-03,1.205e-04, D*=5.99e-02,2.67e-03, lr=1.00e-06]
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:142: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\pde_system.py:143: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


R^2 Error: {'Ni': 0.031, 'SiC': 0.008, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Inf Error: {'Ni': 0.028, 'SiC': 0.017, 'C': inf, 'NiSi': inf, 'NiSi2': inf}
Mean Error: (inf, inf)
模型已保存至 ./Results/AF_PINN_[128]x3/model_save/AF_PINN_[128]x3_10000.pth


e:\Report 12 Some Try\Main_Diffusion_Reaction\graphics.py:137: RuntimeWarning: divide by zero encountered in scalar divide
  relative_R2 = np.linalg.norm(net - fdm, 2)/np.linalg.norm(fdm, 2)
e:\Report 12 Some Try\Main_Diffusion_Reaction\graphics.py:138: RuntimeWarning: divide by zero encountered in scalar divide
  relative_Inf = np.linalg.norm(net - fdm, np.Inf)/np.linalg.norm(fdm, np.Inf)


Training time: 3473.01s-------------------------


In [5]:
# from torch import Tensor


# hiddens = [6, 5, 4, 3]  ##[3,4] already finished
# neurons = [128,96,64]

# activation = torch.nn.functional.silu

# start_lr = 1e-3
# end_lr = 1e-8

# weight = [1, 1, 1, 1, 1, 1]
# for hidden in hiddens:
#     for neuron in neurons:
#         for mode in ['Fourier_PINN']:  ##['PINN', 'IA_PINN', 'Fourier_PINN', 'AF_PINN']
#             layer = [2] + [neuron]*hidden + [6]
            
#             star_time = timeit.default_timer()
#             num_epochs = 5000
#             sub_spochs = 10

#             loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic = [],[],[],[]
#             torch.cuda.empty_cache()
#             print(f'{mode}_[{neuron}]x{hidden}:')
            
#             if mode == 'PINN':
#                 model = PINN(layer,activation,non_negative=True).to(device)
#                 optimizer = optim.Adam(model.parameters(), lr=start_lr)  
#                 scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
#             elif mode == 'IA_PINN':
#                 model = Attention_PINN(layer,activation,non_negative=True).to(device)
#                 optimizer = optim.Adam(model.parameters(), lr=start_lr)  
#                 scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
#             elif mode == 'Fourier_PINN':
#                 model = Fourier_PINN(layer,non_negative=True, activation= activation, use_rff=True, rff_num_features=64, rff_sigma=5.0).to(device)
#                 optimizer = optim.Adam(model.parameters(), lr=start_lr)  
#                 scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
#             elif mode == 'AF_PINN':
#                 model = AF_PINN(layer, non_negative=True,activation = activation, use_rff=True, rff_num_features=64, rff_sigma=5.0).to(device)
#                 optimizer = optim.Adam(model.parameters(), lr=start_lr)  
#                 scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=num_epochs * sub_spochs, eta_min=end_lr)
#             else:
#                 print("Man! What can I say! Wrong Mode!!!")
#                 break            
#             points_types = ['domain', 'left_bc', 'right_bc', 'initial']

#             coords_pde, coords_bc1, coords_bc2, coords_ic = [
#                 generator.grid_generator(2500 if pt in ['left_bc', 'right_bc'] else 10201, points_type=pt, nondim= True) 
#                 for pt in points_types
#             ]

#             for idx in range(sub_spochs):
#                 print(f'Epoch: {idx+1}/{sub_spochs}----------------')
#                 model.train()
                
#                 pbar = tqdm(range(num_epochs))
#                 for epoch in pbar:
#                     current_lr = optimizer.param_groups[0]['lr'] 
#                     def closure() -> Tensor:
#                         global loss_pde, D_star, loss_bc1, loss_bc2, loss_ic1, loss_ic2
#                         optimizer.zero_grad()

#                         # 计算损失时使用batch数据
#                         loss_eq1, loss_eq2, loss_eq3, loss_eq4, loss_eq5, loss_eq6, D_star = compute_pde(model, coords_pde)
#                         loss_pde = loss_eq1 + loss_eq2 + loss_eq3 + loss_eq4 + loss_eq5 + loss_eq6
#                         loss_bc1 = compute_bc(model, coords_bc1)
#                         loss_bc2 = compute_bc(model, coords_bc2)
#                         loss_ic1, loss_ic2 = compute_ic(model, coords_ic)

#                         #Give a weight to initial loss term
#                         loss = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

#                         loss.backward()
#                         torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0, norm_type= 2)
#                         return loss

#                     optimizer.step(closure)
                    
#                     # 如果使用scheduler（只在Adam阶段）
#                     if scheduler is not None:
#                         scheduler.step()
                    
#                     loss_total = weight[0]*loss_pde +  weight[1]*loss_bc1 +  weight[2]*loss_bc2 +  weight[3]*loss_ic1 +  weight[4]*loss_ic2 

#                     loss_log_total.append(loss_total.item())
#                     loss_log_pde.append(loss_pde.item())
#                     loss_log_bc.append(loss_bc1.item() + loss_bc2.item())
#                     loss_log_ic.append(loss_ic1.item() + loss_ic2.item())
                    
#                     if epoch % 10 == 0:
#                         pbar.set_postfix({
#                             'Loss': '{0:.3e}'.format(loss_total),
#                             'PDE': '{0:.3e}'.format(weight[0]*loss_pde.item()),
#                             'BC': '{0:.3e},{1:.3e}'.format(weight[1]*loss_bc1.item(),weight[2]*loss_bc2.item()),
#                             'IC': '{0:.3e},{1:.3e}'.format(weight[3]*loss_ic1.item(),weight[4]*loss_ic2.item()),
#                             'D*': '{0:.2e},{1:.2e}'.format(torch.max(D_star),torch.min(D_star)),
#                             'lr': '{0:.2e}'.format(current_lr),
#                         })        
#                     # # 将四个张量放在一个列表中并随机打乱
#                     shuffled_tensors = [tensor[torch.randperm(tensor.size(0))] for tensor in [coords_pde, coords_bc1, coords_bc2, coords_ic]]

#                     # 解包到四个变量中
#                     coords_pde, coords_bc1, coords_bc2, coords_ic = shuffled_tensors
                
#                     if torch.isnan(loss_total):
#                         print('Loss is Nan, please check your model parameters')
#                         break
                
#                 true_path = './FDM/range[0,1]/'
#                 file_names = ['u1_output.csv','u2_output.csv','u3_output.csv','u4_output.csv','u5_output.csv','u6_output.csv']
#                 true_data = [pd.read_csv(true_path + file_name,header=None) for file_name in file_names]

#                 model.eval()
#                 val_data = generator.grid_generator(N=10201, points_type= 'domain', nondim= True)
#                 result = model(val_data).data.cpu().numpy()
#                 relative_error(result,true_data)

#                 ##模型储存
#                 save_path = f'./Results/{mode}_[{neuron}]x{hidden}/model_save/'
#                 os.makedirs(save_path, exist_ok=True)  # 确保保存路径存在
#                 # 指定模型文件名
#                 model_file = os.path.join(save_path, f'{mode}_[{neuron}]x{hidden}_{(idx+1)*num_epochs}.pth')
#                 # 保存模型
#                 torch.save(model.state_dict(), model_file)
#                 print(f"模型已保存至 {model_file}")
            
#             # 绘制结果
#             plot_loss_curve(f'{mode}_[{neuron}]x{hidden}',loss_log_total,loss_log_pde, loss_log_bc, loss_log_ic)
            
#             plot_relative_concen(f'{mode}_[{neuron}]x{hidden}',result,data_save=True)
            
#             plot_3d_concen(f'{mode}_[{neuron}]x{hidden}', result, true_data)

#             Training_time = timeit.default_timer() - star_time
#             print(f'Training time: {Training_time:.2f}s-------------------------')
